<a href="https://colab.research.google.com/github/Epic721/Understanding-Siri-NLP/blob/main/AK_UnderstandingSiri_N3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Notebook 3
In my third and final notebook, I focused on **NER**, which complemented the sentence-level classification system I started in my second notebook.

In [ ]:
#@title Previously implemented code (N1 & N2)
%tensorflow_version 2.x
%pip install -q transformers

import tensorflow as tf
from urllib.request import urlretrieve
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from transformers import BertTokenizer
from transformers import TFBertModel
from tensorflow.keras.layers import Dropout, Dense
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import SparseCategoricalAccuracy

model_name = "bert-base-cased"
tokenizer = BertTokenizer.from_pretrained(model_name)

# SNIPS_DATA_BASE_URL = (
#     "https://github.com/ogrisel/slot_filling_and_intent_detection_of_SLU/blob/"
#     "master/data/snips/"
# )
# for filename in ["train", "valid", "test", "vocab.intent", "vocab.slot"]:
#     path = Path(filename)
#     if not path.exists():
#       print(f"Downloading {filename}...")
#       urlretrieve(SNIPS_DATA_BASE_URL + filename + "?raw=true", path)

!wget 'https://storage.googleapis.com/inspirit-ai-data-bucket-1/Data/AI%20Scholars/Sessions%206%20-%2010%20(Projects)/Project%20-%20Siri%20(Bert)%20Voice%20Commands/train'
!wget 'https://storage.googleapis.com/inspirit-ai-data-bucket-1/Data/AI%20Scholars/Sessions%206%20-%2010%20(Projects)/Project%20-%20Siri%20(Bert)%20Voice%20Commands/valid'
!wget 'https://storage.googleapis.com/inspirit-ai-data-bucket-1/Data/AI%20Scholars/Sessions%206%20-%2010%20(Projects)/Project%20-%20Siri%20(Bert)%20Voice%20Commands/test'
!wget 'https://storage.googleapis.com/inspirit-ai-data-bucket-1/Data/AI%20Scholars/Sessions%206%20-%2010%20(Projects)/Project%20-%20Siri%20(Bert)%20Voice%20Commands/vocab.intent'
!wget 'https://storage.googleapis.com/inspirit-ai-data-bucket-1/Data/AI%20Scholars/Sessions%206%20-%2010%20(Projects)/Project%20-%20Siri%20(Bert)%20Voice%20Commands/vocab.slot'


def parse_line(line):
    data, intent_label = line.split(" <=> ")
    items = data.split()
    words = [item.rsplit(":", 1)[0]for item in items]
    word_labels = [item.rsplit(":", 1)[1]for item in items]
    return {
        "intent_label": intent_label,
        "words": " ".join(words),
        "word_labels": " ".join(word_labels),
        "length": len(words),
    }

def encode_dataset(text_sequences):
    # Create token_ids array (initialized to all zeros), where
    # rows are a sequence and columns are encoding ids
    # of each token in given sequence.
    token_ids = np.zeros(shape=(len(text_sequences), max_token_len),
                         dtype=np.int32)

    for i, text_sequence in enumerate(text_sequences):
        encoded = tokenizer.encode(text_sequence)
        token_ids[i, 0:len(encoded)] = encoded

    attention_masks = (token_ids != 0).astype(np.int32)
    return {"input_ids": token_ids, "attention_masks": attention_masks}


train_lines = Path("train").read_text().strip().splitlines()
valid_lines = Path("valid").read_text().strip().splitlines()
test_lines = Path("test").read_text().strip().splitlines()

df_train = pd.DataFrame([parse_line(line) for line in train_lines])
df_valid = pd.DataFrame([parse_line(line) for line in valid_lines])
df_test = pd.DataFrame([parse_line(line) for line in test_lines])

max_token_len = 43

encoded_train = encode_dataset(df_train["words"])
encoded_valid = encode_dataset(df_valid["words"])
encoded_test = encode_dataset(df_test["words"])

intent_names = Path("vocab.intent").read_text().split()
intent_map = dict((label, idx) for idx, label in enumerate(intent_names))
intent_train = df_train["intent_label"].map(intent_map).values
intent_valid = df_valid["intent_label"].map(intent_map).values
intent_test = df_test["intent_label"].map(intent_map).values

base_bert_model = TFBertModel.from_pretrained("bert-base-cased")

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 39.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 8.9 MB/s eta 0:00:00


--2023-03-05 19:19:12--  https://storage.googleapis.com/inspirit-ai-data-bucket-1/Data/AI%20Scholars/Sessions%206%20-%2010%20(Projects)/Project%20-%20Siri%20(Bert)%20Voice%20Commands/train
Resolving storage.googleapis.com (storage.googleapis.com)... 172.253.63.128, 142.251.163.128, 142.250.31.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.253.63.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1793794 (1.7M) [application/octet-stream]
Saving to: ‘train’

train               100%[===================>]   1.71M  --.-KB/s    in 0.01s   

2023-03-05 19:19:12 (167 MB/s) - ‘train’ saved [1793794/1793794]

--2023-03-05 19:19:12--  https://storage.googleapis.com/inspirit-ai-data-bucket-1/Data/AI%20Scholars/Sessions%206%20-%2010%20(Projects)/Project%20-%20Siri%20(Bert)%20Voice%20Commands/valid
Resolving storage.googleapis.com (storage.googleapis.com)... 172.253.63.128, 142.251.163.128, 142.250.31.128, ...
Connecting to storage.googleapis.c

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


## Intent Classification + NER

I began to refine my Natural Language Understanding system by capturing the important named elements within each voice command.

To do this, I did word (actually *token*) level classification of the BIO labels as planned.

Since I had *word* level tags but BERT used a tokenizer, I needed to align the BIO labels with the BERT *tokens*.

First, I loaded the list of possible word token labels and augmented it with an additional padding label so I could ignore special tokens:

In [ ]:
# Building a map from slot name to a unique id.
slot_names = ["[PAD]"] + Path("vocab.slot").read_text().strip().splitlines()
slot_map = {}
for label in slot_names:
    slot_map[label] = len(slot_map)
slot_map

{'[PAD]': 0,
 'B-album': 1,
 'B-artist': 2,
 'B-best_rating': 3,
 'B-city': 4,
 'B-condition_description': 5,
 'B-condition_temperature': 6,
 'B-country': 7,
 'B-cuisine': 8,
 'B-current_location': 9,
 'B-entity_name': 10,
 'B-facility': 11,
 'B-genre': 12,
 'B-geographic_poi': 13,
 'B-location_name': 14,
 'B-movie_name': 15,
 'B-movie_type': 16,
 'B-music_item': 17,
 'B-object_location_type': 18,
 'B-object_name': 19,
 'B-object_part_of_series_type': 20,
 'B-object_select': 21,
 'B-object_type': 22,
 'B-party_size_description': 23,
 'B-party_size_number': 24,
 'B-playlist': 25,
 'B-playlist_owner': 26,
 'B-poi': 27,
 'B-rating_unit': 28,
 'B-rating_value': 29,
 'B-restaurant_name': 30,
 'B-restaurant_type': 31,
 'B-served_dish': 32,
 'B-service': 33,
 'B-sort': 34,
 'B-spatial_relation': 35,
 'B-state': 36,
 'B-timeRange': 37,
 'B-track': 38,
 'B-year': 39,
 'I-album': 40,
 'I-artist': 41,
 'I-city': 42,
 'I-country': 43,
 'I-cuisine': 44,
 'I-current_location': 45,
 'I-entity_name': 

#### Word to Token Encodings

Then, I wrote the following function to generate *token-aligned* integer ids from the BIO *word-level* annotations.

If a certain word was broken down into multiple tokens by BERT, the word-level label was replicated for all of the word's tokens. The "B-" prefix was only used for the first of the tokens, while the rest of the tokens had the same label but with the "I-" prefix.



In [ ]:
# Using the slot_map of slot name to unique id (defined above) as well as the BERT tokenizer to create a np array
# with each row corresponding to a given sequence & the columns as the id of the given token slot labels

def encode_token_labels(text_sequences, true_word_labels):
    encoded = np.zeros(shape=(len(text_sequences), max_token_len), dtype=np.int32)
    for i, (text_sequence, word_labels) in enumerate( \
            zip(text_sequences, true_word_labels)):
        encoded_labels = []
        for word, word_label in zip(text_sequence.split(), word_labels.split()): #note: debugged tuple iteration
            tokens = tokenizer.tokenize(word)
            encoded_labels.append(slot_map[word_label])
            expand_label = word_label.replace("B-", "I-")
            if not expand_label in slot_map:
                expand_label = word_label
            encoded_labels.extend([slot_map[expand_label]] * (len(tokens) - 1))
        encoded[i, 1:len(encoded_labels) + 1] = encoded_labels
    return encoded

I then proceeded to encode the token labels for train, validation, & test:

In [ ]:
slot_train = encode_token_labels(df_train["words"], df_train["word_labels"])
slot_valid = encode_token_labels(df_valid["words"], df_valid["word_labels"])
slot_test = encode_token_labels(df_test["words"], df_test["word_labels"])

As an example, this is what the encoded token labels for the first training sequence looks like:

In [ ]:
slot_train[0]

array([ 0, 72, 72, 10, 46, 46, 46, 72, 26, 25, 60, 60, 60, 60, 60, 60, 72,
       72,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0], dtype=int32)

As a reminder, all special tokens such as `[PAD]` and `[SEP]` as well as all padded positions were assigned to have a 0 label.

After appropriately encoding all the labels, I coded up a **joint sequence and token classification model**, which was trained on the encoded dataset with the NER labels:


In [ ]:
# Defining the class for the model that will create predictions
# for the overall intent of a sequence, as well as the NER token labels
class JointIntentAndSlotFillingModel(tf.keras.Model):

    def __init__(self, intent_num_labels=None, slot_num_labels=None,
                dropout_prob=0.1):
        super().__init__(name="joint_intent_slot")

        self.bert = base_bert_model

        # TODO: define the dropout, intent & slot classifier layers
        self.dropout = Dropout(dropout_prob)
        self.intent_classifier = Dense(intent_num_labels, name="intent_classifier")
        self.slot_classifier = Dense(slot_num_labels, name="slot_classifier")

    def call(self, inputs, **kwargs):
        # Extracting features from the inputs using pre-trained BERT.
        #Note: Bert model returns pooled output and individual token output
        tokens_output, pooled_output = self.bert(inputs, **kwargs, return_dict=False)

        # Using the new layers to predict slot class (logits) for each
        # token position in input sequence (size: (batch_size, seq_len, slot_num_labels))
        tokens_output = self.dropout(tokens_output, training=kwargs.get("training", False))
        slot_logits = self.slot_classifier(tokens_output)

        # Defining a second classification head for the sequence-wise predictions (size: (batch_size, intent_num_labels))

        # Reminder for self: second output of the main BERT layer is size
        # (batch_size, output_dim) & gives a "pooled" representation for
        # full sequence from hidden state corresponding to [CLS]).

        # Creating pooled_output to get the intent_logits
        pooled_output = self.dropout(pooled_output, training=kwargs.get("training", False))
        intent_logits = self.intent_classifier(pooled_output)

        return slot_logits, intent_logits

# Creating an instantiation of the model
joint_model = JointIntentAndSlotFillingModel( \
    intent_num_labels=len(intent_map), slot_num_labels=len(slot_map))


In [ ]:
# Defining one classification loss for each output (intent & NER):
losses = [SparseCategoricalCrossentropy(from_logits=True),
          SparseCategoricalCrossentropy(from_logits=True)]

joint_model.compile(optimizer=Adam(learning_rate=3e-5, epsilon=1e-08),
                    loss=losses,
                    metrics=[SparseCategoricalAccuracy('accuracy')], run_eagerly=True)

In [ ]:
# Training the model
history = joint_model.fit(encoded_train["input_ids"], (slot_train, intent_train), \
    validation_data=(encoded_valid["input_ids"], (slot_valid, intent_valid)), \
    epochs=1, batch_size=32)

409/409 [==============================] - 398s 871ms/step - loss: 0.5997 - output_1_loss: 0.3148 - output_2_loss: 0.2849 - output_1_accuracy: 0.9301 - output_2_accuracy: 0.9021 - val_loss: 0.1463 - val_output_1_loss: 0.0583 - val_output_2_loss: 0.0881 - val_output_1_accuracy: 0.9845 - val_output_2_accuracy: 0.9700


Accuracy Check: 99% validation accuracy for both tasks (sequence & token predictions) after only training for one epoch

#### Classification

After the impelementation and training of the joint model, I implemented the following function, which uses the trained model to make a prediction on a single text sequence and display both the sequence-wise and the token-wise class labels:


In [ ]:
# Using the model we trained to get the intent & slot logits and print the actual string of the class corresponding to
# the highest logit score for each token, and the sentence overall

def show_predictions(text, intent_names, slot_names):
    inputs = tf.constant(tokenizer.encode(text))[None, :]  # batch_size = 1
    outputs = joint_model(inputs)
    slot_logits, intent_logits = outputs
    slot_ids = slot_logits.numpy().argmax(axis=-1)[0, 1:-1]
    intent_id = intent_logits.numpy().argmax(axis=-1)[0]
    print("## Intent:", intent_names[intent_id])
    print("## Slots:")
    for token, slot_id in zip(tokenizer.tokenize(text), slot_ids):
        print(f"{token:>10} : {slot_names[slot_id]}")

These are some examples of my classification model in action:

In [ ]:
show_predictions("Book a table for two at Le Ritz for Friday night!", intent_names, slot_names)

## Intent: BookRestaurant
## Slots:
      Book : O
         a : O
     table : O
       for : O
       two : B-party_size_number
        at : O
        Le : B-restaurant_name
         R : I-restaurant_name
     ##itz : I-restaurant_name
       for : O
    Friday : B-timeRange
     night : O
         ! : O


In [ ]:
show_predictions("Will it snow tomorrow in Saclay?", intent_names, slot_names)

## Intent: GetWeather
## Slots:
      Will : O
        it : O
      snow : B-condition_description
  tomorrow : B-timeRange
        in : O
        Sa : B-city
       ##c : I-city
     ##lay : I-city
         ? : O


In [ ]:
show_predictions("I would like to listen to Anima by Thom Yorke.", intent_names, slot_names)

## Intent: PlayMusic
## Slots:
         I : O
     would : O
      like : O
        to : O
    listen : O
        to : O
        An : B-object_name
     ##ima : I-track
        by : O
      Thom : O
      York : I-artist
       ##e : I-artist
         . : O


### Turning Predictions into Structured Knowledge

For completeness, I implemented the following functions to turn the predicted BIO token ids and intent id into a nice, structured representation:

In [ ]:
def decode_predictions(text, intent_names, slot_names,
                       intent_id, slot_ids):
    info = {"intent": intent_names[intent_id]}
    collected_slots = {}
    active_slot_words = []
    active_slot_name = None
    for word in text.split():
        tokens = tokenizer.tokenize(word)
        current_word_slot_ids = slot_ids[:len(tokens)]
        slot_ids = slot_ids[len(tokens):]
        current_word_slot_name = slot_names[current_word_slot_ids[0]]
        if current_word_slot_name == "O":
            if active_slot_name:
                collected_slots[active_slot_name] = " ".join(active_slot_words)
                active_slot_words = []
                active_slot_name = None
        else:
            # Naive BIO: handling: treat B- and I- the same...
            new_slot_name = current_word_slot_name[2:]
            if active_slot_name is None:
                active_slot_words.append(word)
                active_slot_name = new_slot_name
            elif new_slot_name == active_slot_name:
                active_slot_words.append(word)
            else:
                collected_slots[active_slot_name] = " ".join(active_slot_words)
                active_slot_words = [word]
                active_slot_name = new_slot_name
    if active_slot_name:
        collected_slots[active_slot_name] = " ".join(active_slot_words)
    info["slots"] = collected_slots
    return info

In [ ]:
def nlu(text, intent_names, slot_names):
    inputs = tf.constant(tokenizer.encode(text))[None, :]  # batch_size = 1
    outputs = joint_model(inputs)
    slot_logits, intent_logits = outputs
    slot_ids = slot_logits.numpy().argmax(axis=-1)[0, 1:-1]
    intent_id = intent_logits.numpy().argmax(axis=-1)[0]

    return decode_predictions(text, intent_names, slot_names, intent_id, slot_ids)

This is the output of the function on the same examples as before:

In [ ]:
nlu("Book a table for two at Le Ritz for Friday night", intent_names, slot_names)

{'intent': 'BookRestaurant',
 'slots': {'party_size_number': 'two', 'restaurant_name': 'Le Ritz'}}

In [ ]:
nlu("Will it snow tomorrow in Saclay", intent_names, slot_names)

{'intent': 'GetWeather',
 'slots': {'condition_description': 'snow',
  'timeRange': 'tomorrow',
  'city': 'Saclay'}}

In [ ]:
nlu("I would like to listen to Anima by Thom Yorke", intent_names, slot_names)

{'intent': 'PlayMusic',
 'slots': {'object_name': 'Anima', 'artist': 'Thom Yorke'}}